In [1]:
# STEP 1: SETUP & IMPORTS

!pip3 install torchsummaryX

import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F


In [ ]:
# STEP 2: LOAD DATASET
# MNIST dataset

train_dataset = datasets.MNIST(root='./data',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data',
                              train=False,
                              transform=transforms.ToTensor())

# Data loader
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=64,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=64,
                         shuffle=False)


In [ ]:
# STEP 3: ACTIVATE DEVICE

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.get_device_name(0)


In [ ]:
# STEP 4: DEFINE NEURAL NETWORK MODEL (option A - Fully connected Neural Network)
# Fully connected neural network with one hidden layer

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(784, 500)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(500, 10)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        # no activation and no softmax at the end
        return out

model = NeuralNet()
model.to(device)

print (model)

In [ ]:
# STEP 4: DEFINE NEURAL NETWORK MODEL (option B - Convolutional Neural Network)
# LeNET

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = LeNet()
model.to(device)

print (model)

In [ ]:
# Optional: Show the number of parameters in the network

def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

print ("# net parameters: ", get_n_params(model))

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print ("# net parameters: ", count_parameters(model))


from torchsummaryX import summary

if isinstance(model, NeuralNet):
    summary(model, torch.zeros((1, 28*28)).to(device))

if isinstance(model, LeNet):
    summary(model, torch.zeros((1, 1, 28, 28)).to(device))


In [ ]:
# STEP 5: DEFINE LOSS FUNCTION AND OPTIMIZATION METHOD

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# STEP 6: TRAIN THE MODEL

n_total_steps = len(train_loader)
num_epochs = 3

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # images from dataset: [64, 1, 28, 28]
        # labels from dataset: [64]
        #print (images.size())
        #print (labels.size())

        if isinstance(model, NeuralNet):
            images = images.reshape(-1, 28*28)

        # resized: [64, 784]
        #print (images.size())

        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        # outputs: [64, 10]
        #print (outputs.size())
        loss = criterion(outputs, labels)

        # Show kernels
        """
        if isinstance(model, LeNet):
            print (model.conv1.weight.size())
            print (model.conv1.weight)
        """
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')


In [ ]:
# STEP 7: TEST THE MODEL

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:

        if isinstance(model, NeuralNet):
            images = images.reshape(-1, 28*28)

        images, labels = images.to(device), labels.to(device)

        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')